In [1]:
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

plt.style.use('ggplot')
from transformers import  BertTokenizer,DistilBertTokenizer, RobertaModel, RobertaTokenizer
from transformers import  AdamW, get_linear_schedule_with_warmup
import torch

from torch import nn
from sklearn.model_selection import train_test_split
import os
import sys



In [2]:
import sys
import os
#sys.path.append(os.path.abspath("./BERTFamily.py"))
import BERTFamily as fn

In [3]:

RANDOM_SEED = 177
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
MAX_LENGTH = 64
BATCH_SIZE = 16
NUM_CLASSES = 3  # neutral, positive, negative
EPOCHS = 25
DROPOUT_PROB = 0.1
WEIGHT_DECAY = 0.01
NFOLDS = 5
LEARNING_RATE = 1e-4
le = LabelEncoder()
#
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device=torch.device('cpu')
#
loss_function = nn.CrossEntropyLoss().to(device)


# DATA

##  Dataset 1 
all-data.csv
use to train sentiment models specifically on financial news headline. 

In [4]:
statement_df = pd.read_csv('./data/all-data.csv', encoding='latin-1',
                           header=None)
statement_df.columns = ['sentiment', 'statement']
statement_df = statement_df.drop_duplicates()
statement_df['statement'] = statement_df['statement'].apply(fn.clean_statements)
le = LabelEncoder()
statement_df['sentiment'] = le.fit_transform(statement_df['sentiment'])
#positive-2, neutral-1,negative-0
df_train, df_test = train_test_split(statement_df,
                                     test_size=0.2,
                                     random_state=RANDOM_SEED,
                                     stratify=statement_df['sentiment'].values)

df_val, df_test = train_test_split(df_test,
                                   test_size=0.5,
                                   random_state=RANDOM_SEED,
                                   stratify=df_test['sentiment'].values)

df_train_full = pd.concat([df_train, df_val])

## Dataset 2

NASDAQ_headlines.csv
Use NASDAQ news headline as daily news data to predict daily sentiment indices
TIME PERIOD: 2018-3-13 TO 2019-03-28

In [5]:
# head_line_df= pd.read_csv('./data/NASDAQ_headlines.csv',header=None,encoding="utf_8_sig",nrows=10)
# columns_names=["Date","ID","headline","tags"]
# head_line_df.columns=columns_names
# head_line_df.head(5)

## Dataset 3 & 4
observations of daily date about Bitcoin and Ethereum
including ['High', 'Low', 'Open', 'Close', 'Volume', 'tweets', 'Marketcap',
       'google_trends', 'profitability', 'transaction_fee', 'change',
       'fluction']

In [6]:
# BTC_df = pd.read_csv('./data/BTC_Data.csv', encoding='latin-1').iloc[:, 1:]
# BTC_df = BTC_df.set_index(['Date'])

# #BTC_df.isna().sum()
# tweets_mean = BTC_df["tweets"].mean()
# BTC_df["tweets"] = BTC_df["tweets"].fillna(tweets_mean)


In [7]:
# ETH_df = pd.read_csv('./data/ETH_Data.csv', encoding='latin-1').iloc[:, 1:]
# ETH_df = ETH_df.set_index(['Date'])

# #ETH_df.isna().sum()
# Marketcap_mean = ETH_df["Marketcap"].mean()
# ETH_df["Marketcap"] = ETH_df["Marketcap"].fillna(Marketcap_mean)



# SENTIMENT MODEL


## Baseline: LM_dictionary 

In [8]:
# from LM_dictionary import LM_prediction

# train_result = LM_prediction(df_train_full)
# actual = [x for y in [train_result["actual"]] for x in y]
# pres = [x for y in [train_result["prediction"]] for x in y]
# accuracy_score(y_true=actual, y_pred=pres)


In [9]:
# test_result = LM_prediction(df_test)
# actual = [x for y in [test_result["actual"]] for x in y]
# pres = [x for y in [test_result["prediction"]] for x in y]
# accuracy_score(y_true=actual, y_pred=pres)


## Sentiment model 1: BERT

### cased BERT

In [10]:
# %reload_ext autoreload
# %autoreload 1
# %aimport BERTFamily 
#import BERTFamily as fn
import datetime
import json

path = './model/model_' + datetime.datetime.today().strftime('%Y_%m_%d')

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

bert_history, bert_test_outputs = fn.get_oof_and_test_preds(model_type='bert-base-cased',
                                                            tokenizer=bert_tokenizer,
                                                            train_df=df_train_full,
                                                            test_df=df_test,
                                                            single_model=False,
                                                            loss_fn=loss_function)


with open(path + '/bert_history.txt','w') as f:
    f.write(json.dumps(bert_history))


with open (path+'/bert_test_outputs.txt','w') as f:
    f.write(json.dumps(bert_test_outputs))
   

Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.5540732839785585  |  Train Accuracy:  0.12485648679678531
Val Loss:  1.539207842133262  |  Val Accuracy:  0.125
Epoch Train Time:  0:01:54


Epoch  2 / 25
--------------------------------------------------
Train Loss:  1.5028671080913019  |  Train Accuracy:  0.12485648679678531
Val Loss:  1.4596400065855546  |  Val Accuracy:  0.125
Epoch Train Time:  0:01:49


Epoch  3 / 25
--------------------------------------------------
Train Loss:  1.404086186251509  |  Train Accuracy:  0.12485648679678531
Val Loss:  1.3406417239796031  |  Val Accuracy:  0.125
Epoch Train Time:  0:01:47


Epoch  4 / 25
--------------------------------------------------
Train Loss:  1.2750126402312463  |  Train Accuracy:  0.13059701492537312
Val Loss:  1.202814479307695  |  Val Accuracy:  0.125
Epoch Train Time:  0:01:47


early stop
Finished Training.
Fold Train Time:  0:07:18


Fold: 2
Epoch  1 / 25
---------------------------

Train Loss:  0.9116598518069731  |  Train Accuracy:  0.5936872309899569
Val Loss:  0.9051514322107489  |  Val Accuracy:  0.5901262916188289
Epoch Train Time:  0:01:53


Epoch  21 / 25
--------------------------------------------------
Train Loss:  0.9119537735204084  |  Train Accuracy:  0.5908177905308465
Val Loss:  0.9041372526775707  |  Val Accuracy:  0.5889781859931114
Epoch Train Time:  0:01:51


Epoch  22 / 25
--------------------------------------------------
Train Loss:  0.908554763148684  |  Train Accuracy:  0.5954088952654233
Val Loss:  0.9030508594079452  |  Val Accuracy:  0.5901262916188289
Epoch Train Time:  0:01:53


Epoch  23 / 25
--------------------------------------------------
Train Loss:  0.9109436957114333  |  Train Accuracy:  0.5945480631276902
Val Loss:  0.902309122952548  |  Val Accuracy:  0.5889781859931114
Epoch Train Time:  0:01:54


Epoch  24 / 25
--------------------------------------------------
Train Loss:  0.9125421378043813  |  Train Accuracy:  0.5928263

### uncased BERT

In [11]:

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

uncased_bert_history, uncased_bert_test_outputs = fn.get_oof_and_test_preds(model_type='bert-base-uncased',
                                                            tokenizer=bert_tokenizer,
                                                            train_df=df_train_full,
                                                            test_df=df_test,
                                                            single_model=False,
                                                            loss_fn=loss_function)
with open(path +'/uncased_bert_history.txt','w') as f:
    f.write(json.dumps(uncased_bert_history))

with open (path+'/uncased_bert_test_outputs.txt','w') as f:
    f.write(json.dumps(uncased_bert_test_outputs))

Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.2861975333012572  |  Train Accuracy:  0.2804247990815155
Val Loss:  1.2777554793791337  |  Val Accuracy:  0.2809633027522936
Epoch Train Time:  0:02:03


Epoch  2 / 25
--------------------------------------------------
Train Loss:  1.2569746965662054  |  Train Accuracy:  0.28214695752009183
Val Loss:  1.2326851519671354  |  Val Accuracy:  0.2809633027522936
Epoch Train Time:  0:01:50


Epoch  3 / 25
--------------------------------------------------
Train Loss:  1.199890157498351  |  Train Accuracy:  0.28329506314580943
Val Loss:  1.1647798386487094  |  Val Accuracy:  0.2809633027522936
Epoch Train Time:  0:01:48


Epoch  4 / 25
--------------------------------------------------
Train Loss:  1.132187055885245  |  Train Accuracy:  0.29161882893226176
Val Loss:  1.0862048149108887  |  Val Accuracy:  0.2798165137614679
Epoch Train Time:  0:01:51


Epoch  5 / 25
-----------------------------------------

Train Loss:  0.9459493373512128  |  Train Accuracy:  0.5902439024390244
Val Loss:  0.9346320249817588  |  Val Accuracy:  0.5901262916188289
Epoch Train Time:  0:01:51


Epoch  11 / 25
--------------------------------------------------
Train Loss:  0.9414200276956646  |  Train Accuracy:  0.5916786226685796
Val Loss:  0.9321301611987027  |  Val Accuracy:  0.5912743972445464
Epoch Train Time:  0:01:51


Epoch  12 / 25
--------------------------------------------------
Train Loss:  0.9364282469683831  |  Train Accuracy:  0.5936872309899569
Val Loss:  0.9294874158772555  |  Val Accuracy:  0.5924225028702641
Epoch Train Time:  0:01:54


Epoch  13 / 25
--------------------------------------------------
Train Loss:  0.9320142654650801  |  Train Accuracy:  0.5928263988522238
Val Loss:  0.9265944480895996  |  Val Accuracy:  0.5924225028702641
Epoch Train Time:  0:01:50


early stop
Finished Training.
Fold Train Time:  0:24:32


Fold: 5
Epoch  1 / 25
----------------------------------------------

## Sentiment model 2: BERT_large

In [12]:
%reload_ext autoreload
%autoreload 1
%aimport BERTFamily
import BERTFamily as fn
import datetime
import json

path = './model/model_' + datetime.datetime.today().strftime('%Y_%m_%d')

bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
uncased_bert_history, uncased_bert_test_outputs = fn.get_oof_and_test_preds(model_type='bert-large-uncased',
                                                            tokenizer=bert_tokenizer,
                                                            train_df=df_train_full,
                                                            test_df=df_test,
                                                            single_model=False,
                                                            loss_fn=loss_function)
with open(path +'/uncased_bert_large_history.txt','w') as f:
    f.write(json.dumps(uncased_bert_history))

with open (path+'/uncased_bert_large_test_outputs.txt','w') as f:
    f.write(json.dumps(uncased_bert_test_outputs))

Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.4005227034245062  |  Train Accuracy:  0.27525832376578646
Val Loss:  1.3580483133142645  |  Val Accuracy:  0.27522935779816515
Epoch Train Time:  0:06:03


Epoch  2 / 25
--------------------------------------------------
Train Loss:  1.3252914247162846  |  Train Accuracy:  0.27611940298507465
Val Loss:  1.2549495827067982  |  Val Accuracy:  0.27522935779816515
Epoch Train Time:  0:05:37


Epoch  3 / 25
--------------------------------------------------
Train Loss:  1.2051430967969632  |  Train Accuracy:  0.2781285878300804
Val Loss:  1.1141579400409352  |  Val Accuracy:  0.27522935779816515
Epoch Train Time:  0:05:46


Epoch  4 / 25
--------------------------------------------------
Train Loss:  1.0768825753566322  |  Train Accuracy:  0.32893226176808266
Val Loss:  0.9928318370472301  |  Val Accuracy:  0.518348623853211
Epoch Train Time:  0:06:06


Epoch  5 / 25
-------------------------------------

Train Loss:  0.9490476302597501  |  Train Accuracy:  0.5827833572453371
Val Loss:  0.9338024117729881  |  Val Accuracy:  0.5855338691159586
Epoch Train Time:  0:05:22


Epoch  6 / 25
--------------------------------------------------
Train Loss:  0.9423475601804365  |  Train Accuracy:  0.5833572453371593
Val Loss:  0.9280630209229209  |  Val Accuracy:  0.5855338691159586
Epoch Train Time:  0:05:18


early stop
Finished Training.
Fold Train Time:  0:33:21


Fold: 5
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.0124294612932643  |  Train Accuracy:  0.5736011477761837
Val Loss:  0.992143947427923  |  Val Accuracy:  0.5855338691159586
Epoch Train Time:  0:06:01


Epoch  2 / 25
--------------------------------------------------
Train Loss:  0.9996478975912847  |  Train Accuracy:  0.5781922525107605
Val Loss:  0.9713066653771834  |  Val Accuracy:  0.5855338691159586
Epoch Train Time:  0:05:26


Epoch  3 / 25
--------------------------------------------------

In [13]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

uncased_bert_history, uncased_bert_test_outputs = fn.get_oof_and_test_preds(model_type='bert-large-cased',
                                                            tokenizer=bert_tokenizer,
                                                            train_df=df_train_full,
                                                            test_df=df_test,
                                                            single_model=False,
                                                            loss_fn=loss_function)
with open(path +'/cased_bert_history.txt','w') as f:
    f.write(json.dumps(uncased_bert_history))

with open (path+'/cased_bert_test_outputs.txt','w') as f:
    f.write(json.dumps(uncased_bert_test_outputs))

Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.0273017667302298  |  Train Accuracy:  0.574052812858783
Val Loss:  1.0016855998472733  |  Val Accuracy:  0.5848623853211009
Epoch Train Time:  0:05:55


Epoch  2 / 25
--------------------------------------------------
Train Loss:  1.0126900336611162  |  Train Accuracy:  0.5757749712973593
Val Loss:  0.9832518620924516  |  Val Accuracy:  0.5837155963302753
Epoch Train Time:  0:05:26


Epoch  3 / 25
--------------------------------------------------
Train Loss:  1.0009141865126583  |  Train Accuracy:  0.5691733639494834
Val Loss:  0.9629577560858293  |  Val Accuracy:  0.5825688073394495
Epoch Train Time:  0:05:22


Epoch  4 / 25
--------------------------------------------------
Train Loss:  0.9755974029182294  |  Train Accuracy:  0.5760619977037887
Val Loss:  0.9488412250172008  |  Val Accuracy:  0.5837155963302753
Epoch Train Time:  0:05:19


Epoch  5 / 25
-------------------------------------------

Fold: 5
Epoch  1 / 25
--------------------------------------------------
Train Loss:  0.9734773712420682  |  Train Accuracy:  0.5804878048780487
Val Loss:  0.9630644603209062  |  Val Accuracy:  0.5797933409873708
Epoch Train Time:  0:05:45


Epoch  2 / 25
--------------------------------------------------
Train Loss:  0.9656567332941458  |  Train Accuracy:  0.5830703012912483
Val Loss:  0.9511894431981174  |  Val Accuracy:  0.5797933409873708
Epoch Train Time:  0:05:17


Epoch  3 / 25
--------------------------------------------------
Train Loss:  0.9540368190052313  |  Train Accuracy:  0.5807747489239599
Val Loss:  0.9403990106149154  |  Val Accuracy:  0.5797933409873708
Epoch Train Time:  0:05:20


Epoch  4 / 25
--------------------------------------------------
Train Loss:  0.9481695381873244  |  Train Accuracy:  0.5790530846484936
Val Loss:  0.9337999365546487  |  Val Accuracy:  0.5797933409873708
Epoch Train Time:  0:05:24


early stop
Finished Training.
Fold Train Time:  0:21:45


## Sentiment model 3: DistilBERT

In [14]:
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_history, distilbert_test_outputs = fn.get_oof_and_test_preds(model_type='distilbert',
                                                                     tokenizer=distilbert_tokenizer,
                                                                     train_df=df_train_full,
                                                                     test_df=df_test,
                                                                     single_model=False,
                                                                     loss_fn=loss_function)
with open(path +'/distil_history.txt','w') as f:
    f.write(json.dumps(distilbert_history))

with open (path+'/distilbert_test_outputs.txt','w') as f:
    f.write(json.dumps(distilbert_test_outputs))
    

Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  0.9993293933365324  |  Train Accuracy:  0.5777841561423651
Val Loss:  0.9635417266325517  |  Val Accuracy:  0.5940366972477065
Epoch Train Time:  0:02:54


Epoch  2 / 25
--------------------------------------------------
Train Loss:  0.9323762026401835  |  Train Accuracy:  0.5944316877152698
Val Loss:  0.8857188322327354  |  Val Accuracy:  0.5951834862385321
Epoch Train Time:  0:02:52


Epoch  3 / 25
--------------------------------------------------
Train Loss:  0.8310194048312826  |  Train Accuracy:  0.6360505166475315
Val Loss:  0.7497553651983088  |  Val Accuracy:  0.6731651376146789
Epoch Train Time:  0:02:45


Epoch  4 / 25
--------------------------------------------------
Train Loss:  0.6633364480296406  |  Train Accuracy:  0.7146957520091849
Val Loss:  0.5594611140814695  |  Val Accuracy:  0.7706422018348624
Epoch Train Time:  0:02:42


Epoch  5 / 25
------------------------------------------

Train Loss:  0.9390530862392635  |  Train Accuracy:  0.5773314203730272
Val Loss:  0.8690517577258023  |  Val Accuracy:  0.5935706084959816
Epoch Train Time:  0:02:40


Epoch  3 / 25
--------------------------------------------------
Train Loss:  0.8071173131465912  |  Train Accuracy:  0.6321377331420373
Val Loss:  0.7448512380773371  |  Val Accuracy:  0.6739380022962113
Epoch Train Time:  0:02:40


Epoch  4 / 25
--------------------------------------------------
Train Loss:  0.6518034967807455  |  Train Accuracy:  0.7084648493543759
Val Loss:  0.594349455833435  |  Val Accuracy:  0.7324913892078071
Epoch Train Time:  0:02:41


Epoch  5 / 25
--------------------------------------------------
Train Loss:  0.48044533316695365  |  Train Accuracy:  0.8080344332855093
Val Loss:  0.4646176592870192  |  Val Accuracy:  0.7933409873708381
Epoch Train Time:  0:02:40


Epoch  6 / 25
--------------------------------------------------
Train Loss:  0.382410743846259  |  Train Accuracy:  0.8433285509

## Sentiment model 4: RoBERTa 


In [15]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_history, roberta_test_outputs = fn.get_oof_and_test_preds(model_type='roberta',
                                                               tokenizer=roberta_tokenizer,
                                                               train_df=df_train_full,
                                                               test_df=df_test,
                                                               single_model=False,
                                                               loss_fn=loss_function)
with open(path +'/roberta_history.txt','w') as f:
    f.write(json.dumps(roberta_history))

with open (path+'/roberta_test_outputs.txt','w') as f:
    f.write(json.dumps(roberta_test_outputs))


Fold: 1
Epoch  1 / 25
--------------------------------------------------
Train Loss:  1.115650004749998  |  Train Accuracy:  0.20551090700344432
Val Loss:  1.0855396162379871  |  Val Accuracy:  0.5986238532110092
Epoch Train Time:  0:06:00


Epoch  2 / 25
--------------------------------------------------
Train Loss:  1.0097071156589263  |  Train Accuracy:  0.5783582089552238
Val Loss:  0.8990890513766896  |  Val Accuracy:  0.5940366972477065
Epoch Train Time:  0:05:51


Epoch  3 / 25
--------------------------------------------------
Train Loss:  0.8251396120961653  |  Train Accuracy:  0.6214121699196326
Val Loss:  0.7055583829229528  |  Val Accuracy:  0.6731651376146789
Epoch Train Time:  0:05:58


Epoch  4 / 25
--------------------------------------------------
Train Loss:  0.5468624298452237  |  Train Accuracy:  0.7692307692307693
Val Loss:  0.42460769496180795  |  Val Accuracy:  0.8314220183486238
Epoch Train Time:  0:06:03


Epoch  5 / 25
-----------------------------------------

Train Loss:  0.275980267502846  |  Train Accuracy:  0.8946915351506456
Val Loss:  0.3677314291284843  |  Val Accuracy:  0.8495981630309989
Epoch Train Time:  0:06:04


Epoch  7 / 25
--------------------------------------------------
Train Loss:  0.22758438827794628  |  Train Accuracy:  0.914777618364419
Val Loss:  0.4382023071531545  |  Val Accuracy:  0.8495981630309989
Epoch Train Time:  0:05:57


Epoch  8 / 25
--------------------------------------------------
Train Loss:  0.21423389490933045  |  Train Accuracy:  0.9256814921090387
Val Loss:  0.4665228694169359  |  Val Accuracy:  0.8599311136624569
Epoch Train Time:  0:05:59


Epoch  9 / 25
--------------------------------------------------
Train Loss:  0.17452895497356397  |  Train Accuracy:  0.9411764705882353
Val Loss:  0.5656640769405799  |  Val Accuracy:  0.8576349024110218
Epoch Train Time:  0:05:55


early stop
Finished Training.
Fold Train Time:  0:53:43


Fold: 5
Epoch  1 / 25
------------------------------------------------

## Sentiment model 5 : FinBERT

In [16]:
# #sys.path.append(os.path.abspath("./FinBERT.py"))
# from FinBERT import *
# train_results=FinBERT_prediction(data=df_train_full,statement="statement")
# actual = [x for y in [df_train_full["sentiment"]] for x in y]
# pres = [x for y in [train_results["prediction"]] for x in y]
# accuracy_score(y_true=actual, y_pred=pres)

In [17]:
# #test_results=FinBERT_prediction(df_test)

# actual = [x for y in [df_test["sentiment"]] for x in y]
# pres = [x for y in [test_results["prediction"]] for x in y]
# accuracy_score(y_true=actual, y_pred=pres)

# PRICE PREDICTIONN MODEL

## Baseline: VAR

In [19]:
from statsmodels.tsa.api import VAR
from VAR import *
BTC_df = pd.read_csv('./data/BTC_Data.csv', encoding='latin-1').iloc[:, 1:]
BTC_df = BTC_df.set_index(['Date'])
tweets_mean = BTC_df["tweets"].mean()
BTC_df["tweets"] = BTC_df["tweets"].fillna(tweets_mean)
keys = ['Close', 'Volume', 'tweets', 'google_trends','transaction_fee', 'fluction']
targets = ["fluction"]
BTC_df=BTC_df[keys]
#BTC_df = df_differenced[keys]#'change'
split = int(0.7 * len(BTC_df))
train_dataset = BTC_df[0: split - 1]
test_dataset = BTC_df[split:]
VAR_model = VAR(train_dataset)
maxlags = 10
val_model_lag_select=VAR_model.select_order(maxlags=maxlags)
print(val_model_lag_select.summary())

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        71.63       71.67   1.281e+31       71.64
1        58.76       59.03   3.303e+25       58.87
2        58.11       58.62   1.732e+25       58.31
3        57.82      58.57*   1.291e+25      58.11*
4       57.75*       58.73  1.207e+25*       58.13
5        57.76       58.98   1.215e+25       58.23
6        57.76       59.21   1.213e+25       58.32
7        57.77       59.45   1.225e+25       58.42
8        57.75       59.68   1.210e+25       58.50
9        57.76       59.92   1.222e+25       58.60
10       57.79       60.18   1.254e+25       58.71
--------------------------------------------------


In [21]:
VAR_model_fitted1=VAR_model.fit(3)
VAR_model_fitted2=VAR_model.fit(4)

## with lag 3
result=VAR_forecast(VAR_model_fitted1,test_dataset)
flu_prediction=result["fluction"]
flu_actual=test_dataset["fluction"]
a=pd.concat([flu_prediction>0,flu_actual>0],axis=1)
a.columns=['preds','actual']
(a['preds']==a['actual']).sum()/len(a)

0.45874587458745875

In [22]:
## with lag 4
result=VAR_forecast(VAR_model_fitted2,test_dataset)
flu_prediction=result["fluction"]
flu_actual=test_dataset["fluction"]
a=pd.concat([flu_prediction>0,flu_actual>0],axis=1)
a.columns=['preds','actual']
(a['preds']==a['actual']).sum()/len(a)

0.45544554455445546




#### ps: price fluctuation for BTC and ETH would be present in seperate notebooks respectively

